Reads the synthetically generated content file for the context and add them to the question answer pairs

In [1]:
import pandas as pd
df = pd.read_csv("../data/input/hd_fc_by_record.csv")

In [2]:
df_topics =df["filename"].apply(lambda x: x.split("_")[0]).unique()


KeyError: 'filename'

In [6]:
import json
from pathlib import Path
topic_to_content_map= {}
context_file = Path("data/input/discussion_topics_with_llm_content.jsonl")
with open(context_file, "r") as f:
    for line in f:
        data = json.loads(line)
        topic_to_content_map[data["discussion_topic"]] = data["knowledge_base_content"]

In [9]:
set(df_topics).difference(set(topic_to_content_map.keys()))

set()

In [10]:
set(topic_to_content_map.keys()).difference(set(df_topics))

{'The Effects of Noise Pollution on Human Health',
 'The Influence of Art on Society',
 'The significance of rivers in human civilization'}

In [11]:
df["context"] = df["filename"].apply(lambda x: topic_to_content_map[x.split("_")[0]])

In [12]:
df.head()

,filename,hd_score,fc_score,statement,question,answer,context
0,Food Security in Singapore_out_of_topic.json,FAIL,PASS,Food security in Singapore means ensuring that...,I want you to help me understand what food sec...,Food security in Singapore means ensuring that...,# Food Security: A Comprehensive Overview\n\n#...
1,Food Security in Singapore_out_of_topic.json,FAIL,PASS,Singapore ensures a stable food supply through...,"Oh, cool! So since Singapore relies on imports...",Singapore ensures a stable food supply through...,# Food Security: A Comprehensive Overview\n\n#...
2,Food Security in Singapore_out_of_topic.json,FAIL,FAIL,Singapore faces several challenges regarding f...,"That's interesting! But I wonder, what challen...",Singapore faces several challenges regarding f...,# Food Security: A Comprehensive Overview\n\n#...
3,Food Security in Singapore_out_of_topic.json,FAIL,PASS,Climate change impacts food security in Singap...,"Got it! Speaking of climate change, how does t...",Climate change impacts food security in Singap...,# Food Security: A Comprehensive Overview\n\n#...
4,Food Security in Singapore_out_of_topic.json,FAIL,PASS,Local farms in Singapore play a vital role in ...,"Oh wow, that's a lot to think about! So what a...",Local farms in Singapore play a vital role in ...,# Food Security: A Comprehensive Overview\n\n#...


In [24]:
df = df.drop_duplicates(subset=["filename", "question", "answer"])

In [25]:
# overwrite the file with the new context
df.to_csv("data/input/hd_fc_by_record.csv", index=False)